In [ ]:
import os
import pickle
import torch
import torchvision.transforms as transforms
import numpy as np
from flask import Flask, request, jsonify, send_file
from PIL import Image

# ------------ CONFIGURATION ------------
try:
    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    BASE_DIR = os.getcwd()

MODEL_PATH = os.path.join(BASE_DIR, "..", "models", "svm_model.pkl")
FRONTEND_PATH = os.path.join(BASE_DIR, "..", "frontend", "index.html")
UPLOAD_FOLDER = os.path.join(BASE_DIR, "uploads")
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# ------------ FLASK APP INIT ------------
app = Flask(__name__, static_folder="../frontend", template_folder="../frontend")

# ------------ FISH LABELS ------------
fish_labels = {
    0: "Bangus", 1: "Big Head Carp", 2: "Black Spotted Barb", 3: "Catfish", 4: "Climbing Perch",
    5: "Fourfinger Threadfin", 6: "Freshwater Eel", 7: "Glass Perchlet", 8: "Goby", 9: "Gold Fish",
    10: "Gourami", 11: "Grass Carp", 12: "Green Spotted Puffer", 13: "Indian Carp",
    14: "Indo-Pacific Tarpon", 15: "Jaguar Gapote", 16: "Janitor Fish", 17: "Knifefish",
    18: "Long-Snouted Pipefish", 19: "Mosquito Fish", 20: "Mudfish", 21: "Mullet",
    22: "Pangasius", 23: "Perch", 24: "Scat Fish", 25: "Silver Barb", 26: "Silver Carp",
    27: "Silver Perch", 28: "Snakehead", 29: "Tenpounder", 30: "Tilapia"
}

# ------------ LOAD MODEL ------------
print(f"🔍 Loading SVM model from: {MODEL_PATH}")
with open(MODEL_PATH, "rb") as f:
    svm_model = pickle.load(f)

# ------------ LOAD FEATURE EXTRACTOR ------------
print("🔍 Loading ResNet18 feature extractor...")
resnet18 = torch.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=True)
resnet18.fc = torch.nn.Identity()
resnet18.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def extract_features(image):
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = resnet18(image).flatten().numpy()
    return features

# ------------ ROUTES ------------
@app.route("/")
def home():
    return send_file(FRONTEND_PATH)

@app.route("/upload", methods=["POST"])
def upload():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded."}), 400

    file = request.files["image"]
    if file.filename == "":
        return jsonify({"error": "No selected file."}), 400

    save_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(save_path)

    try:
        image = Image.open(save_path).convert("RGB")
        features = extract_features(image)
        prediction = svm_model.predict([features])[0]
        fish_label = fish_labels.get(prediction, "Unknown Fish")

        return jsonify({
            "fish_category": fish_label,
            "image_url": f"/uploads/{file.filename}"
        })

    except Exception as e:
        print(f"[❌] Prediction error: {e}")
        return jsonify({"error": f"Prediction failed: {str(e)}"}), 500

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_file(os.path.join(UPLOAD_FOLDER, filename))

# 🔧 Dummy Chat Endpoint (works offline with fallback answer)
from langchain.chains import RetrievalQA
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_community.llms import HuggingFaceHub

# For fast demo, you can use HuggingFace (or OpenAI if you have key)
from langchain_community.llms import HuggingFaceHub

# ⚠️ Set your Hugging Face token (if needed)
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_token_here"

# Wikipedia wrapper
wiki = WikipediaAPIWrapper(top_k_results=1)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    user_msg = data.get("message", "").strip()

    try:
        if not user_msg:
            return jsonify({"response": "❌ Please ask something."})

        print(f"[🤖] Fetching from Wikipedia for: {user_msg}")
        wiki_summary = wiki.run(user_msg)

        if wiki_summary:
            return jsonify({"response": wiki_summary})
        else:
            return jsonify({"response": "❌ I couldn't find anything useful."})
    except Exception as e:
        print(f"[❌] Chatbot error: {e}")
        return jsonify({"response": "❌ Error contacting Wikipedia bot."}), 500


# ------------ MAIN ------------
if __name__ == "__main__":
    print("🔧 app.py is running...")
    app.run(debug=True, use_reloader=False)


🔍 Loading SVM model from: C:\Users\abina\fish_recognition\backend\..\models\svm_model.pkl
🔍 Loading ResNet18 feature extractor...


Using cache found in C:\Users\abina/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🔧 app.py is running...
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2025 23:06:56] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2025 23:07:07] "POST /upload HTTP/1.1" 200 -


[🤖] Fetching from Wikipedia for: bangus


...

127.0.0.1 - - [17/Jul/2025 23:45:58] "POST /chat HTTP/1.1" 200 -


[🤖] Fetching from Wikipedia for: Gourami


127.0.0.1 - - [17/Jul/2025 23:46:13] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 23:46:42] "POST /upload HTTP/1.1" 200 -


[🤖] Fetching from Wikipedia for: bangus Milkfish


127.0.0.1 - - [17/Jul/2025 23:46:55] "POST /chat HTTP/1.1" 200 -


In [3]:
!pip install langchain langchain-community wikipedia


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
    --------------------------------------- 0.1/2.5 MB 825.8 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.5 MB 1.7 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.5 MB 1.7 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.5 MB 1.7 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.5 MB 1.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.5 MB 1.2 MB/s eta 0:00:02
   ------- -------------------------------- 0.5/2.5 MB 1.3 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.5 MB 1.6 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.5 MB 1.5 MB/s eta 0:00:02
   -------------- ------------------------- 0.9/2.5 MB 1.9 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 1.9 MB/s eta 0:00:01
   ----------------- ---